In [1]:
%reset -f

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers
import os

## Загружаем данные

In [11]:
# Функция для загрузки данных из файлов CSV
def load_data_from_csv(folder_path, num_train_files=9):
    train_data = []
    train_labels = []
    val_data = []
    val_labels = []
    
    files = sorted(os.listdir(folder_path))  # Сортируем файлы для последовательности
    
    # Чтение первых 9 файлов для обучения
    for file in files[:num_train_files]:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        train_data.append(df.iloc[:, :-1].values)  # Признаки
        train_labels.append(df.iloc[:, -1].values)  # Целевая переменная
    
    # Чтение 10-го файла для валидации
    val_file_path = os.path.join(folder_path, files[num_train_files])
    val_df = pd.read_csv(val_file_path)
    val_data = val_df.iloc[:, :-1].values
    val_labels = val_df.iloc[:, -1].values
    
    # Объединяем данные из всех файлов
    train_data = np.vstack(train_data)
    train_labels = np.hstack(train_labels)
    
    return train_data, train_labels, val_data, val_labels

# Загрузка данных
folder_path = r"D:\Programming\international_hack_data\preprocessed_data"  # Путь к папке с CSV файлами
X_train, y_train, X_val, y_val = load_data_from_csv(folder_path)
X_train


KeyboardInterrupt



In [13]:
X_val.shape, y_val.shape

((510800, 39), (510800,))

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_val)

## Models

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, Dropout
from sklearn.metrics import accuracy_score

In [14]:
# Определение функции потерь Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt + tf.keras.backend.epsilon())
    return focal_loss_fixed

In [14]:
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train_scaled.shape[1], 1)),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled[..., np.newaxis], y_train, epochs=1, batch_size=32, verbose=1)

# Оценка модели на валидационных данных
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

  1898/143663 [..............................] - ETA: 2:09:52 - loss: nan - accuracy: 0.0000e+00


KeyboardInterrupt



In [25]:
model1 = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train_scaled.shape[1], 1)),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train_scaled[..., np.newaxis], y_train, epochs=1, batch_size=32, verbose=1)

model2 = Sequential()
model2.add(SimpleRNN(64, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model2.add(SimpleRNN(32, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
# model2.add(Dropout(0.6))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train_scaled[..., np.newaxis], y_train, epochs=1, batch_size=32, verbose=1)

model3 = Sequential()
model3.add(Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train_scaled, y_train, epochs=1, batch_size=32, verbose=1)

 4/56 [=>............................] - ETA: 3:11 - loss: 0.6906 - accuracy: 0.6328

KeyboardInterrupt: 

In [27]:
pred_train1 = model1.predict(X_train_scaled[..., np.newaxis])
pred_train2 = model2.predict(X_train_scaled[..., np.newaxis])
pred_train3 = model3.predict(X_train_scaled)

pred_test1 = model1.predict(X_test_scaled[..., np.newaxis])
pred_test2 = model2.predict(X_test_scaled[..., np.newaxis])
pred_test3 = model3.predict(X_test_scaled)

# Объединение предсказаний для обучения логистической регрессии
stacked_predictions_train = np.hstack([pred_train1, pred_train2, pred_train3])
stacked_predictions_test = np.hstack([pred_test1, pred_test2, pred_test3])
stacked_predictions_train

14/14 [==============================] - 0s 8ms/step


array([[0.5045496 , 0.5408978 , 0.5187068 ],
       [0.508897  , 0.5554257 , 0.57199776],
       [0.27748528, 0.41797104, 0.40897095],
       ...,
       [0.49839798, 0.5991533 , 0.4078643 ],
       [0.28624314, 0.23571089, 0.38419157],
       [0.41224277, 0.49348524, 0.46176916]], dtype=float32)

In [28]:
# Линейная регрессия в качестве финальной модели
final_model = LogisticRegression()
final_model.fit(stacked_predictions_train, y_train)

LogisticRegression()

In [31]:
# Оценка модели
final_predictions = final_model.predict(stacked_predictions_test)
accuracy = accuracy_score(y_val, final_predictions)
print(f'Final Ensemble Model Accuracy: {accuracy:.4f}')

Final Ensemble Model Accuracy: 0.6509
